<a href="https://colab.research.google.com/github/SegunDave/BybitCCxt/blob/main/ByBitData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip -q install ccxt pandas_ta schedule plotly==5.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 KB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [2]:
import ccxt,pytz,time,schedule, requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import clear_output
import matplotlib.pyplot as plt
#from binance.client import Client
from pprint import pprint
import statsmodels.regression.linear_model as rg



In [11]:
bybit = ccxt.bybit()

instrument = 'MATIC/USDT'
Tframe ='15m'
lim = 1000

# get BTCUSDT 1 hr kline data
klines = bybit.fetch_ohlcv(instrument, timeframe=Tframe, limit= lim, since=None)

# filter klines to only include data from the past month
from datetime import datetime, timedelta
one_month_ago = datetime.now() - timedelta(days=30)
filtered_klines = [kline for kline in klines if datetime.fromtimestamp(kline[0]/1000) >= one_month_ago]

print(filtered_klines)


[[1672104600000, 0.8108, 0.8108, 0.8087, 0.8089, 7622.08], [1672105500000, 0.8089, 0.8109, 0.8087, 0.8108, 8582.41], [1672106400000, 0.8108, 0.8136, 0.8106, 0.8127, 12351.46], [1672107300000, 0.8127, 0.8141, 0.8125, 0.8134, 7785.9], [1672108200000, 0.8134, 0.8145, 0.8127, 0.8139, 10199.25], [1672109100000, 0.8139, 0.8144, 0.8136, 0.814, 1470.43], [1672110000000, 0.814, 0.8144, 0.8125, 0.8125, 13337.78], [1672110900000, 0.8125, 0.8135, 0.8119, 0.813, 3479.08], [1672111800000, 0.813, 0.8134, 0.8119, 0.8132, 7938.87], [1672112700000, 0.8132, 0.8138, 0.8128, 0.8137, 3761.76], [1672113600000, 0.8137, 0.8141, 0.8135, 0.8141, 1124.78], [1672114500000, 0.8141, 0.8142, 0.8118, 0.8118, 9023.26], [1672115400000, 0.8118, 0.8122, 0.811, 0.811, 6771.76], [1672116300000, 0.811, 0.8129, 0.811, 0.8126, 6509.83], [1672117200000, 0.8126, 0.8133, 0.8122, 0.8129, 3158.07], [1672118100000, 0.8129, 0.8129, 0.8114, 0.8115, 2690.82], [1672119000000, 0.8115, 0.8125, 0.8115, 0.8117, 1253.89], [1672119900000, 0.8

In [12]:

# convert the klines list to a DataFrame
df = pd.DataFrame(filtered_klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# convert the timestamp column to a datetime type
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# set the timestamp column as the index
df.set_index('timestamp', inplace=True)

print(df)


                       open    high     low   close    volume
timestamp                                                    
2022-12-27 01:30:00  0.8108  0.8108  0.8087  0.8089   7622.08
2022-12-27 01:45:00  0.8089  0.8109  0.8087  0.8108   8582.41
2022-12-27 02:00:00  0.8108  0.8136  0.8106  0.8127  12351.46
2022-12-27 02:15:00  0.8127  0.8141  0.8125  0.8134   7785.90
2022-12-27 02:30:00  0.8134  0.8145  0.8127  0.8139  10199.25
...                     ...     ...     ...     ...       ...
2023-01-06 10:15:00  0.7776  0.7786  0.7775  0.7779   6607.89
2023-01-06 10:30:00  0.7779  0.7787  0.7775  0.7777  58943.93
2023-01-06 10:45:00  0.7777  0.7782  0.7750  0.7757  19535.15
2023-01-06 11:00:00  0.7757  0.7772  0.7725  0.7767  23476.07
2023-01-06 11:15:00  0.7767  0.7767  0.7759  0.7760   2920.44

[1000 rows x 5 columns]


In [13]:
import plotly.graph_objs as go

# create a line chart using the close data
line = go.Scatter(
    x=df.index,
    y=df['close'],
    name='Close',
    line=dict(color='#17BECF')
)

# create the layout for the chart
layout = go.Layout(
    title= f'{instrument} in {Tframe} Kline Data',
    xaxis=dict(
        title='Time',
        rangeslider=dict(visible=False)
    ),
    yaxis=dict(
        title='Price (USDT)'
    )
)

# create the figure and plot the chart
fig = go.Figure(data=[line], layout=layout)
fig.show()

# calculate RSI
df['rsi'] = ta.rsi(df['close'])

# calculate Stochastics
df['stoch_k'], df['stoch_d'] = ta.stoch(df['high'], df['low'], df['close'])


In [15]:
import plotly.express as px

# create the line chart
fig = px.line(df, x=df.index, y=df['rsi'], title=f'{instrument} in {Tframe} Kline Data')

# add the red and green lines
fig.add_shape(
    type='line',
    x0=df.index[0],
    x1=df.index[-1],
    y0=75,
    y1=75,
    yref='y',
    line=dict(color='red', dash='dot')
)
fig.add_shape(
    type='line',
    x0=df.index[0],
    x1=df.index[-1],
    y0=25,
    y1=25,
    yref='y',
    line=dict(color='green', dash='dot')
)

# set the y-axis range to include 0 and 100
fig.update_layout(yaxis=dict(range=[0, 100]))

fig.show()
